<a href="https://colab.research.google.com/github/jihokwak/mlstudy/blob/master/keras/%EC%BC%80%EB%9D%BC%EC%8A%A4%EC%B0%BD%EC%8B%9C%EC%9E%90_%EA%B3%A0%EA%B8%89%EB%8F%84%EA%B5%AC_%ED%95%A8%EC%88%98%ED%98%95API__%EC%97%B0%EC%8A%B56_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import layers

Using TensorFlow backend.


## 특성 맵의 크기가 같을 때 원본 그대로 사용하는 잔차 연결 구현 예

In [0]:
#원본 x에 레이어를 쌓아 변환을 적용
x = layers.Input(shape=(71,71,1))
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

In [14]:
print(x)
print(y)

Tensor("input_4:0", shape=(?, 71, 71, 1), dtype=float32)
Tensor("conv2d_9/Relu:0", shape=(?, 71, 71, 128), dtype=float32)


In [0]:
#원본 x를 출력 특성에 더함
y = layers.add([y,x])

## 특성 맵의 크기가 다를때 선형 변환을 사용하여 잔차 연결 구현한 예

In [0]:
x = layers.Input(shape=(70,70,1))
y = layers.Conv2D(128,3,activation='relu', padding='same')(x)
y = layers.Conv2D(128,3,activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

In [28]:
print(x)
print(y)

Tensor("input_6:0", shape=(?, 70, 70, 1), dtype=float32)
Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 35, 35, 128), dtype=float32)


In [29]:
#y와 크기를 맞추기 위해 1X1 합성곱을 사용하여 원본 텐서 x를 다운샘플링
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
print(residual)

Tensor("conv2d_21/BiasAdd:0", shape=(?, 35, 35, 128), dtype=float32)


In [0]:
#다운샘플링된 x를 출력 특성에 더함
y = layers.add([y,residual])

In [31]:
print(y)

Tensor("add_4/add:0", shape=(?, 35, 35, 128), dtype=float32)


##  ResNet 구현모델 사용해보기

In [0]:
from keras.applications import resnet50

In [0]:
resnet50_model = resnet50.ResNet50(include_top=False, weights=None, input_shape=(32,32,1), pooling='avg', classes=10)

In [37]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 1)    0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
#데이터 준비
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [51]:
#데이터 전처리
from skimage.transform import resize
import numpy as np
X_train = np.asarray([resize(x, (32,32)) for x in X_train]).reshape(-1,32,32,1)
X_test = np.asarray([resize(x, (32,32)) for x in X_test]).reshape(-1,32,32,1)

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [0]:
#모델 최종출력 수정
from keras.models import Model, Sequential
from keras.layers import Dense
add_model = Sequential()
add_model.add(Dense(10, activation='softmax'))

resnet50 = Model(resnet50_model.input, add_model(resnet50_model.output))

In [45]:
resnet50.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 1)    0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [54]:
resnet50.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])
resnet50.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 50s 1ms/step - loss: 0.0478 - acc: 0.9864 - val_loss: 0.0646 - val_acc: 0.9823
Epoch 2/10
48000/48000 [==============================] - 40s 826us/step - loss: 0.0380 - acc: 0.9890 - val_loss: 0.0541 - val_acc: 0.9841
Epoch 3/10
48000/48000 [==============================] - 39s 815us/step - loss: 0.0232 - acc: 0.9928 - val_loss: 0.0392 - val_acc: 0.9888
Epoch 4/10
48000/48000 [==============================] - 40s 824us/step - loss: 0.0169 - acc: 0.9951 - val_loss: 0.0567 - val_acc: 0.9851
Epoch 5/10
48000/48000 [==============================] - 39s 823us/step - loss: 0.0136 - acc: 0.9957 - val_loss: 0.0506 - val_acc: 0.9867
Epoch 6/10
48000/48000 [==============================] - 39s 822us/step - loss: 0.0136 - acc: 0.9962 - val_loss: 0.0495 - val_acc: 0.9880
Epoch 7/10
48000/48000 [==============================] - 39s 816us/step - loss: 0.0131 - acc: 0.9959 - val_loss: 0.04

In [55]:
resnet50.evaluate(X_test, y_test)

10000/10000 [==============================] - 4s 377us/step


[0.054573153910099065, 0.9852]